In [17]:
# Import Libraries Spark
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql import functions as sf, Window as w
from pyspark.sql.functions import udf
from collections import Counter
from pyspark.sql.functions import concat, col, lit, concat_ws, row_number
from pyspark.sql.functions import explode, explode_outer, split, length, trim, mean, min, max
from pyspark.sql.types import StructField, DecimalType, StringType, IntegerType, DoubleType, FloatType, DateType, TimestampType
from pyspark import StorageLevel

In [18]:
# Create Spark Session
spark = SparkSession.builder.master(
    "local[*]").appName("CouncilDataExploration").config('spark.sql.debug.maxToStringFields', 2000).getOrCreate()
input_directory =  "/home/pydev/workflow/dt_learn_data_science/spark/codility/data"   

In [19]:
# district_councils
district_councilsdf = spark.read.option("header", "true").option("inferSchema", "true").option(
        "sep", ",").csv(f"{input_directory}/england_councils/district_councils.csv")
df1 = district_councilsdf.withColumn("council_type", lit('district councils')) 

# londonboroughs
london_boroughsdf = spark.read.option("header", "true").option("inferSchema", "true").option(
        "sep", ",").csv(f"{input_directory}/england_councils/london_boroughs.csv")
df2 = london_boroughsdf.withColumn("council_type", lit('londonboroughs')) 

# metropolitandistricts
metropolitan_districtsdf = spark.read.option("header", "true").option("inferSchema", "true").option(
        "sep", ",").csv(f"{input_directory}/england_councils/metropolitan_districts.csv")
df3 = metropolitan_districtsdf.withColumn("council_type", lit('metropolitandistricts')) 


# unitary_authorities
unitary_authoritiesdf = spark.read.option("header", "true").option("inferSchema", "true").option(
        "sep", ",").csv(f"{input_directory}/england_councils/unitary_authorities.csv")
df4 = unitary_authoritiesdf.withColumn("council_type", lit('unitary authorities')) 

council_df = df1.unionAll(df2)
council_df = council_df.unionAll(df3)
council_df = council_df.unionAll(df4)


In [20]:
# avg_price_nov_2019
unitary_authoritiesdf = spark.read.option("header", "true").option("inferSchema", "true")\
        .option("sep", ",").csv(f"{input_directory}/property_avg_price.csv") \
        .select(col('local_authority').alias('council'), col('avg_price_nov_2019'))
# sales_volume_sep_2019      
property_sales_volumedf = spark.read.option("header", "true").option("inferSchema", "true")\
        .option("sep", ",").csv(f"{input_directory}/property_sales_volume.csv")\
        .select(col('local_authority').alias('council'), col('sales_volume_sep_2019'))        

In [24]:
### Union Data
final_df = council_df.join(unitary_authoritiesdf, council_df.council == unitary_authoritiesdf.council, "left")\
        .select(council_df.council, council_df.county, council_df.council_type, unitary_authoritiesdf.avg_price_nov_2019)
# final_df.show()
final_df = final_df.join(property_sales_volumedf,final_df.council == property_sales_volumedf.council,"left")\
        .select(final_df.council,final_df.county,final_df.council_type,final_df.avg_price_nov_2019,property_sales_volumedf.sales_volume_sep_2019)    


In [27]:
x = final_df.count()
print(x)

316


In [28]:
final_df.show()

+--------------------+---------------+-----------------+------------------+---------------------+
|             council|         county|     council_type|avg_price_nov_2019|sales_volume_sep_2019|
+--------------------+---------------+-----------------+------------------+---------------------+
|                Adur|    West Sussex|district councils|          316482.0|                   82|
|           Allerdale|        Cumbria|district councils|          148708.0|                  122|
|        Amber Valley|     Derbyshire|district councils|          184204.0|                  156|
|                Arun|    West Sussex|district councils|          290304.0|                  234|
|            Ashfield|Nottinghamshire|district councils|          141916.0|                  156|
|             Ashford|           Kent|district councils|          304770.0|                  137|
|      Aylesbury Vale|Buckinghamshire|district councils|          321553.0|                  245|
|             Baberg